# Stock Ticker Symbol Data cleaning

The ticker symbol data is downloaded from [NASDAQ.com](http://www.nasdaq.com/screening/company-list.aspx). For this project we would like to select stocks with marketcapital larger than 10 billion.

In [38]:
import requests
import pandas as pd
import numpy as np
import time
import feather
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### Download ticker symbols data for US exchanges

In [39]:
exchanges = ["NASDAQ", "NYSE", "AMEX"]

for exchange in exchanges:

    # construct the url to download data
    url_NASDAQ = "http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=" + exchange + "&render=download"
    
    # web scrapping
    readdata = requests.get(url_NASDAQ , allow_redirects = True)

    # write to a file
    file = open("../data/data_Ticker/" + exchange + ".csv", 'wb')
    file.write(readdata.content)
    file.close()

471352

432903

49963

### Read data 

In [40]:
# read ticker symbols from all US stocks, data source (http://www.nasdaq.com/screening/company-list.aspx)
tickers_NASDAQ = pd.read_csv("../data/data_Ticker/NASDAQ.csv", skiprows=None, )
tickers_NASDAQ ["Exchange"] = "NASDAQ"
#tickers_NASDAQ = tickers_NASDAQ.drop("Unnamed: 8", axis=1)

tickers_NYSE = pd.read_csv("../data/data_Ticker/NYSE.csv", skiprows=None)
tickers_NYSE ["Exchange"] = "NYSE"
#tickers_NYSE = tickers_NYSE.drop("Unnamed: 8", axis=1)

tickers_AMEX = pd.read_csv("../data/data_Ticker/AMEX.csv", skiprows=None)
tickers_AMEX ["Exchange"] = "AMEX"
#tickers_AMEX = tickers_AMEX.drop("Unnamed: 8", axis=1)

# combine all symbols from three exchanges into one dataframe
tickers = tickers_NASDAQ.append(tickers_NYSE)
tickers = tickers.append(tickers)

In [21]:
tickers.head(5)

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Exchange
0,PIH,"1347 Property Insurance Holdings, Inc.",7.35,4.398803e+07,n/a,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN,NASDAQ
1,TURN,180 Degree Capital Corp.,2.04,6.348799e+07,n/a,n/a,Finance,Finance/Investors Services,http://www.nasdaq.com/symbol/turn,NaN,NASDAQ
2,FLWS,"1-800 FLOWERS.COM, Inc.",10.5,6.785569e+08,n/a,1999,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws,NaN,NASDAQ
3,FCCY,1st Constitution Bancorp (NJ),18.15,1.465278e+08,n/a,n/a,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy,NaN,NASDAQ
4,SRCE,1st Source Corporation,52.93,1.372799e+09,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN,NASDAQ


### Clean data
There are many duplicated symbols, uncommon symbols, and one symbol with multiple classes of stocks. For example, all symbols for "Bank of America Corporation" is shown below.

In [22]:
tickers.loc[tickers.Name == "Bank of America Corporation"]

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Exchange
321,BAC,Bank of America Corporation,28.28,2.949778e+11,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/bac,NaN,NYSE
322,BAC.WS.A,Bank of America Corporation,n/a,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac.ws.a,NaN,NYSE
323,BAC.WS.B,Bank of America Corporation,n/a,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac.ws.b,NaN,NYSE
324,BAC^A,Bank of America Corporation,26.91,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac^a,NaN,NYSE
325,BAC^C,Bank of America Corporation,27.02,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac^c,NaN,NYSE
326,BAC^D,Bank of America Corporation,25.7,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac^d,NaN,NYSE
327,BAC^E,Bank of America Corporation,23.28,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac^e,NaN,NYSE
328,BAC^I,Bank of America Corporation,26.7455,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac^i,NaN,NYSE
329,BAC^L,Bank of America Corporation,1325.7111,0.000000e+00,n/a,n/a,n/a,n/a,http://www.nasdaq.com/symbol/bac^l,NaN,NYSE
330,BAC^W,Bank of America Corporation,26.66,0.000000e+00,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/bac^w,NaN,NYSE


__Data cleaning:__
    
* remove symbols with `^` 
* reomve symbols without stock price `LastSale == n/a`
* reomve symbols without market captical `MarketCap == n/a`

In [23]:
# remove stocks with ^ signs
data_size = tickers.shape
data_size = np.arange(data_size[0])   # give a integer vector for indexing in tickers.
tickers = tickers.reset_index()     # reset index after joining multiple dataframes
tickers.drop(list(data_size[tickers.Symbol.str.contains("\^").tolist()]),axis = 0, inplace=True)
tickers = tickers.reset_index()    # reset index after deleting some rows

# drop colums created when reseting index.
tickers.drop(["level_0","index"], axis=1, inplace=True)  

# reomve symbols without market captical `MarketCap == n/a`
tmp = tickers.shape
tmp = np.arange(tmp[0])
tickers.drop(list(tmp[tickers.MarketCap==0]),axis = 0, inplace=True)
tickers = tickers.reset_index()

# delete duplicated names
tickers.drop_duplicates(["Name"], inplace=True)

# delete rows with Sector == n/a
tmp = tickers.shape
tmp = np.arange(tmp[0])
tickers = tickers.reset_index()
tickers.drop(["level_0","index"], axis=1, inplace=True)
tickers.drop(list(tmp[tickers.Sector=="n/a"]),axis = 0, inplace=True)
tickers = tickers.reset_index()

# drop colums created when reseting index.
tickers.drop(["index"], axis=1, inplace=True)
tickers.head(5)

tickers.describe()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Exchange
0,PIH,"1347 Property Insurance Holdings, Inc.",7.35,4.398803e+07,n/a,2014,Finance,Property-Casualty Insurers,http://www.nasdaq.com/symbol/pih,NaN,NASDAQ
1,TURN,180 Degree Capital Corp.,2.04,6.348799e+07,n/a,n/a,Finance,Finance/Investors Services,http://www.nasdaq.com/symbol/turn,NaN,NASDAQ
2,FLWS,"1-800 FLOWERS.COM, Inc.",10.5,6.785569e+08,n/a,1999,Consumer Services,Other Specialty Stores,http://www.nasdaq.com/symbol/flws,NaN,NASDAQ
3,FCCY,1st Constitution Bancorp (NJ),18.15,1.465278e+08,n/a,n/a,Finance,Savings Institutions,http://www.nasdaq.com/symbol/fccy,NaN,NASDAQ
4,SRCE,1st Source Corporation,52.93,1.372799e+09,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN,NASDAQ


,MarketCap,Unnamed: 9
count,4.507000e+03,0.0
mean,8.581739e+09,NaN
std,3.362471e+10,NaN
min,8.522400e+05,NaN
25%,2.103232e+08,NaN
50%,9.996746e+08,NaN
75%,4.341243e+09,NaN
max,8.701632e+11,NaN


In [27]:
tickers.dtypes

Symbol            object
Name              object
LastSale          object
MarketCap        float64
ADR TSO           object
IPOyear           object
Sector            object
Industry          object
Summary Quote     object
Unnamed: 9       float64
Exchange          object
dtype: object

In [24]:
tickers.loc[tickers.Name == "Bank of America Corporation"]

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Exchange
2733,BAC,Bank of America Corporation,28.28,2.949778e+11,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/bac,NaN,NYSE


### Select tickers with only market cap larger than 10 Billion

Here we only select stocks with market cap that is larger than 10 billion in 2017-11. To do that, we can first select rows in which `MarketCap` column contains `B` (billion). Then covert the type to numeric and selecting values larger than 10.

In [31]:
# select ticker symbols with `MarketCap` contains "B"
tickers_Billion = tickers.loc[tickers.MarketCap>1000000000]
tickers_Billion.describe()

# see the filtered symbols
tickers_Billion = tickers_Billion.reset_index();
tickers_Billion.drop(["index"], axis=1, inplace=True)
tickers_Billion.head(5)

,MarketCap,Unnamed: 9
count,2.253000e+03,0.0
mean,1.685824e+10,NaN
std,4.609919e+10,NaN
min,1.002263e+09,NaN
25%,2.102459e+09,NaN
50%,4.342748e+09,NaN
75%,1.245546e+10,NaN
max,8.701632e+11,NaN


,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Exchange
0,SRCE,1st Source Corporation,52.93,1.372799e+09,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN,NASDAQ
1,TWOU,"2U, Inc.",63.98,3.341186e+09,n/a,2014,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/twou,NaN,NASDAQ
2,JOBS,"51job, Inc.",58.22,2.142110e+09,36793371,2004,Technology,Diversified Commercial Services,http://www.nasdaq.com/symbol/jobs,NaN,NASDAQ
3,CAFD,8point3 Energy Partners LP,14.92,1.179947e+09,n/a,2015,Public Utilities,Electric Utilities: Central,http://www.nasdaq.com/symbol/cafd,NaN,NASDAQ
4,EGHT,8x8 Inc,14.025,1.289634e+09,n/a,n/a,Public Utilities,Telecommunications Equipment,http://www.nasdaq.com/symbol/eght,NaN,NASDAQ


In [32]:
# convert last sale price type to numeric
tickers_Billion.loc[:,"LastSale"] = tickers_Billion.loc[:,"LastSale"].astype(float);

# # remove $ and B in `MarketCap`
# tmp = tickers_Billion.MarketCap.str.replace("$", "")
# tmp = tmp.str.replace("B", "")
# #tmp.reset_index()

# # change marketcap to real numercial values.
# tickers_Billion.MarketCap= tmp.astype(float) * 1000000000

tickers_Billion.head()

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Exchange
0,SRCE,1st Source Corporation,52.930,1.372799e+09,n/a,n/a,Finance,Major Banks,http://www.nasdaq.com/symbol/srce,NaN,NASDAQ
1,TWOU,"2U, Inc.",63.980,3.341186e+09,n/a,2014,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/twou,NaN,NASDAQ
2,JOBS,"51job, Inc.",58.220,2.142110e+09,36793371,2004,Technology,Diversified Commercial Services,http://www.nasdaq.com/symbol/jobs,NaN,NASDAQ
3,CAFD,8point3 Energy Partners LP,14.920,1.179947e+09,n/a,2015,Public Utilities,Electric Utilities: Central,http://www.nasdaq.com/symbol/cafd,NaN,NASDAQ
4,EGHT,8x8 Inc,14.025,1.289634e+09,n/a,n/a,Public Utilities,Telecommunications Equipment,http://www.nasdaq.com/symbol/eght,NaN,NASDAQ


__Select Market Cap larger than 10 billions__

In [34]:
tickers_TenBillion = tickers_Billion.loc[tickers_Billion.MarketCap > 50000000000]

tickers_TenBillion = tickers_TenBillion.reset_index()
tickers_TenBillion.drop(["index"], axis=1, inplace=True)

tickers_TenBillion.head()
tickers_TenBillion.shape

,Symbol,Name,LastSale,MarketCap,ADR TSO,IPOyear,Sector,Industry,Summary Quote,Unnamed: 9,Exchange
0,ADBE,Adobe Systems Incorporated,176.8500,8.717697e+10,n/a,1986,Technology,Computer Software: Prepackaged Software,http://www.nasdaq.com/symbol/adbe,NaN,NASDAQ
1,GOOG,Alphabet Inc.,1021.6600,7.098515e+11,n/a,2004,Technology,"Computer Software: Programming, Data Processing",http://www.nasdaq.com/symbol/goog,NaN,NASDAQ
2,AABA,Altaba Inc.,70.4300,6.151202e+10,n/a,n/a,Technology,EDP Services,http://www.nasdaq.com/symbol/aaba,NaN,NASDAQ
3,AMZN,"Amazon.com, Inc.",1160.5500,5.592368e+11,n/a,1997,Consumer Services,Catalog/Specialty Distribution,http://www.nasdaq.com/symbol/amzn,NaN,NASDAQ
4,AMOV,"America Movil, S.A.B. de C.V.",17.5524,5.797295e+10,n/a,n/a,Public Utilities,Telecommunications Equipment,http://www.nasdaq.com/symbol/amov,NaN,NASDAQ


(173, 11)

In [35]:
tickers_TenBillion.Exchange.describe()

count      173
unique       2
top       NYSE
freq       137
Name: Exchange, dtype: object

#### Save the ticker symbols dataframe to feature (for read in R)

In [11]:
# Write to a csv file
tickers_TenBillion.to_csv("../data/data_Ticker/tickers_TenBillion.csv")

# Write to a feather file for R
feather.write_dataframe(tickers_TenBillion, "../data/data_Ticker/tikers_TenBillion.feather")